In [1]:
import pandas as pd
import numpy as np
import ast
import json

In [2]:
with open('australian_users_items.json', encoding='utf-8') as a:
    items = a.readlines()
items[0]
j = ast.literal_eval(items[0])
# Create a string representing a list with each line seperated by comma
newstring = '[' + ','.join(items) + ']'

# Evaluate this new string
j = ast.literal_eval(newstring)

# Save file as JSON
with open('data.json', 'w') as json_file:
    json.dump(j, json_file)
    
#We now have a .json file that we can easily view as a Pandas DataFrame.
# Load dataframe to check
df_items = pd.DataFrame(j)
df_items.head(10)

,user_id,items_count,steam_id,user_url,items
0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
1,js41637,888,76561198035864385,http://steamcommunity.com/id/js41637,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
2,evcentric,137,76561198007712555,http://steamcommunity.com/id/evcentric,"[{'item_id': '1200', 'item_name': 'Red Orchest..."
3,Riot-Punch,328,76561197963445855,http://steamcommunity.com/id/Riot-Punch,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
4,doctr,541,76561198002099482,http://steamcommunity.com/id/doctr,"[{'item_id': '300', 'item_name': 'Day of Defea..."
5,MinxIsBetterThanPotatoes,371,76561198004744620,http://steamcommunity.com/id/MinxIsBetterThanP...,"[{'item_id': '50', 'item_name': 'Half-Life: Op..."
6,NitemarePK,304,76561197990951820,http://steamcommunity.com/id/NitemarePK,"[{'item_id': '240', 'item_name': 'Counter-Stri..."
7,themanwich,258,76561198035296505,http://steamcommunity.com/id/themanwich,"[{'item_id': '220', 'item_name': 'Half-Life 2'..."
8,maplemage,629,76561198026584251,http://steamcommunity.com/id/maplemage,"[{'item_id': '240', 'item_name': 'Counter-Stri..."
9,Wackky,0,76561198039117046,http://steamcommunity.com/id/Wackky,[]


In [5]:
numgames = df_items[['user_id', 'items_count']]
numgames.head()
#numgames.to_csv('numgames.csv')

,user_id,items_count
0,76561197970982479,277
1,js41637,888
2,evcentric,137
3,Riot-Punch,328
4,doctr,541


In [6]:
# Get all item_id for first user
gameids = [df_items['items'][0][index]['item_id'] for index, _ in enumerate(df_items['items'][0])]
# Show first 10 item ids
gameids[:10]

['10', '20', '30', '40', '50', '60', '70', '130', '300', '240']

In [7]:
# Create column with item ids
df_items['item_id'] = df_items['items'].apply(lambda x: [x [index]['item_id'] for index, _ in enumerate(x)])
df_items.head()

,user_id,items_count,steam_id,user_url,items,item_id
0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"[{'item_id': '10', 'item_name': 'Counter-Strik...","[10, 20, 30, 40, 50, 60, 70, 130, 300, 240, 38..."
1,js41637,888,76561198035864385,http://steamcommunity.com/id/js41637,"[{'item_id': '10', 'item_name': 'Counter-Strik...","[10, 80, 100, 300, 30, 40, 60, 240, 280, 360, ..."
2,evcentric,137,76561198007712555,http://steamcommunity.com/id/evcentric,"[{'item_id': '1200', 'item_name': 'Red Orchest...","[1200, 1230, 1280, 1520, 220, 320, 340, 360, 3..."
3,Riot-Punch,328,76561197963445855,http://steamcommunity.com/id/Riot-Punch,"[{'item_id': '10', 'item_name': 'Counter-Strik...","[10, 20, 30, 40, 50, 60, 70, 130, 80, 100, 300..."
4,doctr,541,76561198002099482,http://steamcommunity.com/id/doctr,"[{'item_id': '300', 'item_name': 'Day of Defea...","[300, 20, 50, 70, 130, 10, 30, 40, 60, 80, 100..."


As the unique user steam_id is a large integer, we will replace it with a new uid counter, which starts at 0 and increments by 1 (like the index).

We will also only select the relevant columns for the purpose of building a user-item interactions matrix, namely the newly created user id iud and the item_id.

In [9]:
# Add a column with substitute user_id, counter
df_items['uid'] = np.arange(len(df_items))

# Take relevant columns
df_items = df_items[['uid', 'item_id']]

# Check
df_items.head()

,uid,item_id
0,0,"[10, 20, 30, 40, 50, 60, 70, 130, 300, 240, 38..."
1,1,"[10, 80, 100, 300, 30, 40, 60, 240, 280, 360, ..."
2,2,"[1200, 1230, 1280, 1520, 220, 320, 340, 360, 3..."
3,3,"[10, 20, 30, 40, 50, 60, 70, 130, 80, 100, 300..."
4,4,"[300, 20, 50, 70, 130, 10, 30, 40, 60, 80, 100..."


In [10]:
# Explode item_ids into seperate rows
lst_col = 'item_id'
df_items = pd.DataFrame({col:np.repeat(df_items[col].values, df_items[lst_col].str.len())
                              for col in df_items.columns.difference([lst_col])
                        }).assign(**{lst_col:np.concatenate(df_items[lst_col].values)})[df_items.columns.tolist()]
df_items

,uid,item_id
0,0,10
1,0,20
2,0,30
3,0,40
4,0,50
...,...,...
5153204,88308,346330
5153205,88308,373330
5153206,88308,388490
5153207,88308,521570


In [11]:
# Add binary owned column
df_items['owned'] = np.ones(shape = df_items.shape[0])

# Check
df_items.head()

,uid,item_id,owned
0,0,10,1.0
1,0,20,1.0
2,0,30,1.0
3,0,40,1.0
4,0,50,1.0


In [12]:
# Change item_id to int
df_items['item_id'] = df_items['item_id'].astype(int)

# Rename column to match
df_items = df_items.rename(columns={'item_id': 'id'})

In [13]:
with open('steam_games.json', encoding='utf-8') as b:
    data  = b.readlines()
data[0]
j = ast.literal_eval(data[0])
# Create a string representing a list with each line seperated by comma
newstring = '[' + ','.join(data) + ']'

# Evaluate this new string
j = ast.literal_eval(newstring)

# Save file as JSON
with open('data.json', 'w') as json_file:
    json.dump(j, json_file)
    
#We now have a .json file that we can easily view as a Pandas DataFrame.
# Load dataframe to check
df_data = pd.DataFrame(j)
df_data.head(5)

,publisher,genres,app_name,title,url,release_date,tags,discount_price,reviews_url,specs,price,early_access,id,developer,sentiment,metascore
0,Kotoshiro,"[Action, Casual, Indie, Simulation, Strategy]",Lost Summoner Kitty,Lost Summoner Kitty,http://store.steampowered.com/app/761140/Lost_...,2018-01-04,"[Strategy, Action, Indie, Casual, Simulation]",4.49,http://steamcommunity.com/app/761140/reviews/?...,[Single-player],4.99,False,761140,Kotoshiro,NaN,NaN
1,"Making Fun, Inc.","[Free to Play, Indie, RPG, Strategy]",Ironbound,Ironbound,http://store.steampowered.com/app/643980/Ironb...,2018-01-04,"[Free to Play, Strategy, Indie, RPG, Card Game...",NaN,http://steamcommunity.com/app/643980/reviews/?...,"[Single-player, Multi-player, Online Multi-Pla...",Free To Play,False,643980,Secret Level SRL,Mostly Positive,NaN
2,Poolians.com,"[Casual, Free to Play, Indie, Simulation, Sports]",Real Pool 3D - Poolians,Real Pool 3D - Poolians,http://store.steampowered.com/app/670290/Real_...,2017-07-24,"[Free to Play, Simulation, Sports, Casual, Ind...",NaN,http://steamcommunity.com/app/670290/reviews/?...,"[Single-player, Multi-player, Online Multi-Pla...",Free to Play,False,670290,Poolians.com,Mostly Positive,NaN
3,彼岸领域,"[Action, Adventure, Casual]",弹炸人2222,弹炸人2222,http://store.steampowered.com/app/767400/2222/,2017-12-07,"[Action, Adventure, Casual]",0.83,http://steamcommunity.com/app/767400/reviews/?...,[Single-player],0.99,False,767400,彼岸领域,NaN,NaN
4,NaN,NaN,Log Challenge,NaN,http://store.steampowered.com/app/773570/Log_C...,NaN,"[Action, Indie, Casual, Sports]",1.79,http://steamcommunity.com/app/773570/reviews/?...,"[Single-player, Full controller support, HTC V...",2.99,False,773570,NaN,NaN,NaN


In [18]:
df_data = pd.read_csv('gamesdata.csv')

In [19]:
# Merge useritems and games data dataframes
alldata = pd.merge(df_items, df_data, on = 'id')
alldata.head()

,uid,id,owned,Unnamed: 0,publisher,genres,app_name,title,url,release_date,tags,discount_price,reviews_url,specs,price,early_access,developer,sentiment,metascore
0,0,10,1.0,32106,Valve,['Action'],Counter-Strike,Counter-Strike,http://store.steampowered.com/app/10/CounterSt...,2000-11-01,"['Action', 'FPS', 'Multiplayer', 'Shooter', 'C...",NaN,http://steamcommunity.com/app/10/reviews/?brow...,"['Multi-player', 'Valve Anti-Cheat enabled']",9.99,False,Valve,Overwhelmingly Positive,88.0
1,1,10,1.0,32106,Valve,['Action'],Counter-Strike,Counter-Strike,http://store.steampowered.com/app/10/CounterSt...,2000-11-01,"['Action', 'FPS', 'Multiplayer', 'Shooter', 'C...",NaN,http://steamcommunity.com/app/10/reviews/?brow...,"['Multi-player', 'Valve Anti-Cheat enabled']",9.99,False,Valve,Overwhelmingly Positive,88.0
2,3,10,1.0,32106,Valve,['Action'],Counter-Strike,Counter-Strike,http://store.steampowered.com/app/10/CounterSt...,2000-11-01,"['Action', 'FPS', 'Multiplayer', 'Shooter', 'C...",NaN,http://steamcommunity.com/app/10/reviews/?brow...,"['Multi-player', 'Valve Anti-Cheat enabled']",9.99,False,Valve,Overwhelmingly Positive,88.0
3,4,10,1.0,32106,Valve,['Action'],Counter-Strike,Counter-Strike,http://store.steampowered.com/app/10/CounterSt...,2000-11-01,"['Action', 'FPS', 'Multiplayer', 'Shooter', 'C...",NaN,http://steamcommunity.com/app/10/reviews/?brow...,"['Multi-player', 'Valve Anti-Cheat enabled']",9.99,False,Valve,Overwhelmingly Positive,88.0
4,10,10,1.0,32106,Valve,['Action'],Counter-Strike,Counter-Strike,http://store.steampowered.com/app/10/CounterSt...,2000-11-01,"['Action', 'FPS', 'Multiplayer', 'Shooter', 'C...",NaN,http://steamcommunity.com/app/10/reviews/?brow...,"['Multi-player', 'Valve Anti-Cheat enabled']",9.99,False,Valve,Overwhelmingly Positive,88.0


In [20]:
# Drop entries with no title
datawithnames = alldata.dropna(axis=0, subset=['title'])
datawithnames.to_csv('mergeddata.csv')
datawithnames.head()

,uid,id,owned,Unnamed: 0,publisher,genres,app_name,title,url,release_date,tags,discount_price,reviews_url,specs,price,early_access,developer,sentiment,metascore
0,0,10,1.0,32106,Valve,['Action'],Counter-Strike,Counter-Strike,http://store.steampowered.com/app/10/CounterSt...,2000-11-01,"['Action', 'FPS', 'Multiplayer', 'Shooter', 'C...",NaN,http://steamcommunity.com/app/10/reviews/?brow...,"['Multi-player', 'Valve Anti-Cheat enabled']",9.99,False,Valve,Overwhelmingly Positive,88.0
1,1,10,1.0,32106,Valve,['Action'],Counter-Strike,Counter-Strike,http://store.steampowered.com/app/10/CounterSt...,2000-11-01,"['Action', 'FPS', 'Multiplayer', 'Shooter', 'C...",NaN,http://steamcommunity.com/app/10/reviews/?brow...,"['Multi-player', 'Valve Anti-Cheat enabled']",9.99,False,Valve,Overwhelmingly Positive,88.0
2,3,10,1.0,32106,Valve,['Action'],Counter-Strike,Counter-Strike,http://store.steampowered.com/app/10/CounterSt...,2000-11-01,"['Action', 'FPS', 'Multiplayer', 'Shooter', 'C...",NaN,http://steamcommunity.com/app/10/reviews/?brow...,"['Multi-player', 'Valve Anti-Cheat enabled']",9.99,False,Valve,Overwhelmingly Positive,88.0
3,4,10,1.0,32106,Valve,['Action'],Counter-Strike,Counter-Strike,http://store.steampowered.com/app/10/CounterSt...,2000-11-01,"['Action', 'FPS', 'Multiplayer', 'Shooter', 'C...",NaN,http://steamcommunity.com/app/10/reviews/?brow...,"['Multi-player', 'Valve Anti-Cheat enabled']",9.99,False,Valve,Overwhelmingly Positive,88.0
4,10,10,1.0,32106,Valve,['Action'],Counter-Strike,Counter-Strike,http://store.steampowered.com/app/10/CounterSt...,2000-11-01,"['Action', 'FPS', 'Multiplayer', 'Shooter', 'C...",NaN,http://steamcommunity.com/app/10/reviews/?brow...,"['Multi-player', 'Valve Anti-Cheat enabled']",9.99,False,Valve,Overwhelmingly Positive,88.0


In [21]:
# Get relevant columns for recommendation engine
recdata = datawithnames[['uid','id','owned']]
# Save to csv
recdata.to_csv('recdata.csv')

recdata.head()

,uid,id,owned
0,0,10,1.0
1,1,10,1.0
2,3,10,1.0
3,4,10,1.0
4,10,10,1.0
